Combined alg

In [168]:
import scipy.stats as stats
import heapq
from collections import defaultdict
from datetime import datetime, timedelta
import pandas as pd
from data_preparation import prepare_data,import_data
import time
pd.set_option('display.max_colwidth', None)

In [32]:
agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df,calendar_dates_df = import_data()

In [83]:
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

def get_available_service_ids(start_date, calendar, calendar_dates):
    """
    Gibt eine Liste der verfügbaren Service-IDs für das angegebene Datum zurück.
    """
    # Konvertiere start_date zu einem Datum im Format YYYYMMDD
    start_date_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    start_date_str = start_date_datetime.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_date_datetime.weekday()]

    available_service_ids = []
    
    
        # 🔹 Schritt 2: Reguläre Services aus `calendar` prüfen
    for _, service in calendar.iterrows():
        service_id = service["service_id"]

        # Prüfen, ob das Datum im gültigen Zeitraum liegt
        if int(service["start_date"]) <= int(start_date_str) <= int(service["end_date"]):
            # Prüfen, ob der Service an diesem Wochentag aktiv ist
            if service[weekday] == 1:
                if service_id not in available_service_ids:
                    available_service_ids.append(service_id)
                   

    # 🔹 Schritt 1: Sonderregelungen aus `calendar_dates` berücksichtigen
    exceptions = calendar_dates[calendar_dates["date"] == int(start_date_str)]
    
    for _, exception in exceptions.iterrows():
        service_id = exception["service_id"]
        if exception["exception_type"] == 2:  # Service wird als Ausnahme hinzugefügt
            if service_id not in available_service_ids:
                available_service_ids.append(service_id)
        elif exception["exception_type"] == 1:  # Service wird als Ausnahme entfernt
            if service_id in available_service_ids:
                available_service_ids.remove(service_id)
    return available_service_ids 


    

def prepare_calendar_dates(calendar_dates):
    grouped = calendar_dates.groupby("service_id")
    calendar_dates_dict = {}
    for service_id, group in grouped:
        exceptions = group.to_dict(orient="records")
        calendar_dates_dict[service_id] = exceptions
    return calendar_dates_dict

Create Graph

In [147]:
def create_graph_with_schedule(start_time_obj, end_time_obj):

    from collections import defaultdict

    graph = defaultdict(list)
    stop_id_to_name = stops_df.set_index("stop_id")["stop_name"].to_dict()

    # 🔹 Hole alle gültigen Service-IDs für den Tag
    available_services = get_available_service_ids(start_time_obj.strftime("%Y-%m-%d"), calendar_df, calendar_dates_df)
    print(len(available_services))

    # 🔹 Filtere `stop_times` basierend auf der Zeitspanne
    stop_times = stop_times_df.sort_values(by=["trip_id", "stop_sequence"])
    stop_times["arrival_minutes"] = stop_times["arrival_time"].apply(time_to_minutes)
    stop_times["departure_minutes"] = stop_times["departure_time"].apply(time_to_minutes)
    trip_id_to_service = trips_df.set_index("trip_id")["service_id"].to_dict()
    stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times_filtered = stop_times[
        (stop_times["arrival_minutes"] >= start_minutes) &
        (stop_times["departure_minutes"] <= end_minutes) &
        (stop_times["service_id"].isin(available_services))
    ]

    print(f"Rows after time window and service filter: {len(stop_times_filtered)}")

    # 🔹 Graphen erstellen
    trip_id_to_route = trips_df.set_index("trip_id")["route_id"].to_dict()
    grouped = stop_times_filtered.groupby("trip_id")

    for trip_id, group in grouped:
        stops_list = group["stop_id"].tolist()
        departures = group["departure_time"].apply(time_to_minutes).tolist()
        arrivals = group["arrival_time"].apply(time_to_minutes).tolist()

        for start, end, dep, arr in zip(stops_list[:-1], stops_list[1:], departures[:-1], arrivals[1:]):
            travel_time = arr - dep
            if travel_time > 0:
                graph[stop_id_to_name[start]].append(
                    (stop_id_to_name[end], dep, arr, trip_id_to_route.get(trip_id))
                )

    return graph


Reliability function

In [148]:
def compute_transfer_probability_with_departure_delay(transfer_time):
    return min(0.95,stats.gamma.cdf(transfer_time, a=2, scale=2))


In [149]:
def compute_total_reliability(reliability_fast, backup_routes):
    total_reliability = reliability_fast
    #remaining_probability = 1 - reliability_fast  # Wahrscheinlichkeit, dass die Primärroute fehlschlägt

    for backup in backup_routes:
        backup_reliability = backup[2]
        total_reliability += backup_reliability

    return total_reliability
#zwischen 0 und 1


# A* 

In [150]:
import heapq
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius der Erde in Kilometern
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Entfernung in Kilometern

def heuristic(current_stop, end_name):
    current_stop_row = stops_df[stops_df['stop_name'] == current_stop].iloc[0]
    end_stop_row = stops_df[stops_df['stop_name'] == end_name].iloc[0]

    current_lat = current_stop_row['stop_lat']
    current_lon = current_stop_row['stop_lon']
    end_lat = end_stop_row['stop_lat']
    end_lon = end_stop_row['stop_lon']

    distance_km = haversine(current_lat, current_lon, end_lat, end_lon)

    average_speed = 60  # km/h
    estimated_time_minutes = (distance_km / average_speed) * 60
    return estimated_time_minutes


Backups

In [163]:
def a_star_with_reliability_fixed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=4):
    pq = [(start_time_minutes,start_time_minutes, start_name, [], 1.0, None)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern
    # Initialisierung für die Überprüfung der Verbesserungen
    count = 0  # Zähler für Iterationen ohne Verbesserung
   
    
    while pq:
        weight, current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)

        path_hash = tuple(path[-5:])
        if (current_stop, current_time, path_hash) in visited:
            continue
        visited.add((current_stop, current_time,path_hash))

        path = path + [(current_stop, current_time)]

        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            print("possibility")
            count += 1
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or reliability > best_result[2]:
                print("possibility is better then before", best_result)
                # Wenn das Ergebnis besser ist, wird es aktualisiert
                best_result = (current_time, path, reliability) # Zähler zurücksetzen
                
        
            # Überprüfung, ob maximale Anzahl an Iterationen ohne Verbesserung erreicht wurde
        if count > 2:
            print("max ieration for here")
            return best_result  # Rückgabe des aktuellen besten Ergebnisses


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME or transfer_time > 60:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time+h ,# Gesamtbewertung: aktuelle Zeit + Heuristik
                    new_current_time,
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id
                    
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden


Main

In [194]:
def a_star_on_speed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2,time_limit = 600):
    pq = [(start_time_minutes,start_time_minutes, start_name, [], 1.0, None)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    results = []
    start_time = time.time()
    
    while pq:
        if time.time() - start_time > time_limit and len(results) > 0:
            print("Time limit exceeded. Terminating search.")
            break  # Exit the loop if time limit is exceeded
         
        weight, current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)
        path_hash = tuple(path[-5:])# todo changr 

        if (current_stop, current_time,path_hash) in visited:
            continue
        visited.add((current_stop, current_time, path_hash))

        path = path + [(current_stop, current_time)]

        # Check if the goal is reached
        if current_stop == end_name:
            print("dest reached")
            results.append((current_time, path, reliability))
            results = sorted(results, key=lambda x: x[0])[:15]  # Keep only the top 10 results
            if reliability == 1:
                return results
            if len(results) == 15:
                return results 

        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    #print(transfer_time)
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)
                    #print(transfer_reliability)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability
                #print(new_reliability,reliability,transfer_reliability)
                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    current_time+h, # Gesamtbewertung: aktuelle Zeit + Heuristik
                    new_current_time,  
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id
                    
                ))
                
                
                
    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if results:
        return results
    else:
        return []  # No route found




In [195]:
def find_path(start_stop_name,end_stop_name,start_datetime,time_budget):
    
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + time_budget
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    MRIB_reliability = 0
    MRIB = None
    MRIB_Backups = None
    MRIB_time = 100000

    graph = create_graph_with_schedule(start_time_obj, end_time_obj)
    completed_primary_trips = a_star_on_speed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=2)
    transfer_dict = {}
    
    for trips in completed_primary_trips:
        current_time_fast = trips[0]
        primary_itinerary = trips[1]
        reliability_fast = trips[2]
        print("here",primary_itinerary)
        print(current_time_fast,reliability_fast)

        
        Backups = []
        last_route = None
        rel_before_trans = 1.0

        for i in range(1, len(primary_itinerary) - 1, 2):  # Alle Umstiegspunkte durchgehen
            transfer_stop, current  = primary_itinerary[i - 1]
            route_id, departure_time, arrival_time = primary_itinerary[i]
            next_stop, next_time = primary_itinerary[i + 1]
            transfer_time_inbetween = departure_time - current 
            
            if route_id != last_route and transfer_stop != start_stop_name:

                print(f"lets go from{transfer_stop}")
                rel_before_trans *= (compute_transfer_probability_with_departure_delay(transfer_time_inbetween))
                missed_trans_rel = 1 - (compute_transfer_probability_with_departure_delay(transfer_time_inbetween))
                
                transfer = (
                    transfer_stop, current, next_stop, next_time)      
                              
                if transfer not in transfer_dict.keys():
                    backup_time, backup_path, backup_reliability = a_star_with_reliability_fixed(
                        graph, transfer_stop, primary_itinerary[-1][0], start_time_minutes = departure_time + 1 , MIN_TRANSFER_TIME= 2,exclude_routes=set()
                    )

                    transfer_dict[transfer] = backup_time, backup_path, backup_reliability
                else: 
                    print("reuse backup")
                    backup_time, backup_path, backup_reliability = transfer_dict.get(transfer)
                if backup_path:             
                    Backups.append([backup_time, backup_path, (missed_trans_rel * backup_reliability * rel_before_trans)])
            last_route = route_id
            
        total_reliability =compute_total_reliability(reliability_fast, Backups)
        if round(total_reliability, 4) > MRIB_reliability or (round(total_reliability, 4) == MRIB_reliability and current_time_fast < MRIB_time):
                MRIB_reliability = round(total_reliability, 4)
                MRIB = primary_itinerary
                MRIB_Backups = Backups
                MRIB_time = current_time_fast
                print(MRIB_reliability,MRIB)
                
    return MRIB_reliability, MRIB, MRIB_Backups


In [196]:
start_stop_name = "Wien Leopoldau" #"Laa/Thaya Bahnhof" #"Schattendorf Kirchengasse" #"Gmunden Bahnhof" # for now the departure node is 1
end_stop_name = "Laa/Thaya Bahnhof" #"Wien Leopoldau" #"    Flughafen Wien Bahnhof" #Prinzersdorf Bahnhof"
start_datetime = "2024-12-12 09:00:00"
time_budget = timedelta(hours=1, minutes = 30)

In [197]:
find_path(start_stop_name,end_stop_name,start_datetime,time_budget)

1907
Rows after time window and service filter: 13380
dest reached
here [('Wien Leopoldau', 540), ('2-RX2-W-j24-1', 543.0, 551.0), ('Obersdorf Bahnhof', 551.0), ('2-RX2-W-j24-1', 551.0, 553.0), ('Wolkersdorf Bahnhof', 553.0), ('2-RX2-W-j24-1', 555.0, 577.0), ('Mistelbach Bahnhof', 577.0), ('2-RX2-W-j24-1', 578.0, 579.0), ('Mistelbach Stadt', 579.0), ('2-RX2-W-j24-1', 579.0, 584.0), ('Siebenhirten (NÖ) Haltestelle', 584.0), ('2-RX2-W-j24-1', 585.0, 586.0), ('Hörersdorf Bahnhof', 586.0), ('2-RX2-W-j24-1', 587.0, 590.0), ('Frättingsdorf Bahnhof', 590.0), ('2-RX2-W-j24-1', 590.0, 594.0), ('Enzersdorf bei Staatz Bahnhof', 594.0), ('2-RX2-W-j24-1', 595.0, 598.0), ('Staatz Bahnhof', 598.0), ('2-RX2-W-j24-1', 598.0, 600.0), ('Kottingneusiedl Bahnhof', 600.0), ('2-RX2-W-j24-1', 601.0, 605.0), ('Laa/Thaya Bahnhof', 605.0)]
605.0 1.0
1.0 [('Wien Leopoldau', 540), ('2-RX2-W-j24-1', 543.0, 551.0), ('Obersdorf Bahnhof', 551.0), ('2-RX2-W-j24-1', 551.0, 553.0), ('Wolkersdorf Bahnhof', 553.0), ('2-RX2

(1.0,
 [('Wien Leopoldau', 540),
  ('2-RX2-W-j24-1', 543.0, 551.0),
  ('Obersdorf Bahnhof', 551.0),
  ('2-RX2-W-j24-1', 551.0, 553.0),
  ('Wolkersdorf Bahnhof', 553.0),
  ('2-RX2-W-j24-1', 555.0, 577.0),
  ('Mistelbach Bahnhof', 577.0),
  ('2-RX2-W-j24-1', 578.0, 579.0),
  ('Mistelbach Stadt', 579.0),
  ('2-RX2-W-j24-1', 579.0, 584.0),
  ('Siebenhirten (NÖ) Haltestelle', 584.0),
  ('2-RX2-W-j24-1', 585.0, 586.0),
  ('Hörersdorf Bahnhof', 586.0),
  ('2-RX2-W-j24-1', 587.0, 590.0),
  ('Frättingsdorf Bahnhof', 590.0),
  ('2-RX2-W-j24-1', 590.0, 594.0),
  ('Enzersdorf bei Staatz Bahnhof', 594.0),
  ('2-RX2-W-j24-1', 595.0, 598.0),
  ('Staatz Bahnhof', 598.0),
  ('2-RX2-W-j24-1', 598.0, 600.0),
  ('Kottingneusiedl Bahnhof', 600.0),
  ('2-RX2-W-j24-1', 601.0, 605.0),
  ('Laa/Thaya Bahnhof', 605.0)],
 [])

In [185]:
minutes_to_time(646)

'10:46'